In [82]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [83]:
dc = pd.read_csv('DecOp_data_EN_500.csv', sep=',', encoding='UTF-8')#encoding='ISO-8859-1')

d1 = []
for i in range(dc.shape[0]):
  row = dc.iloc[i]
  d1.append({'ID': row['ID'],
      'age' :row['age'], 'gender': row['gender'],
  'sent' : row['A'].replace('\n', " ") ,
  'labels'  : row['GT.A']})

  d1.append({'ID': row['ID'],
      'age' :row['age'], 'gender': row['gender'],
  'sent' : row['E'].replace('\n', " ")  ,
  'labels'  : row['GT.E']})
  d1.append({'ID': row['ID'],
      'age' :row['age'], 'gender': row['gender'],
  'sent' : row['GM'].replace('\n', " ")  ,
  'labels'  : row['GT.GM']})

  d1.append({'ID': row['ID'],
      'age' :row['age'], 'gender': row['gender'],
  'sent' : row['Pom'].replace('\n', " ")  ,
  'labels'  : row['GT.Pom']})

  d1.append({'ID': row['ID'],
      'age' :row['age'], 'gender': row['gender'],
  'sent' : row['CL'].replace('\n', " ")  ,
  'labels'  : row['GT.CL']})

decop = pd.DataFrame.from_records(d1)

decop = decop[['ID', 'sent', 'labels']]
decop['type'] = 'A'

decop

,ID,sent,labels,type
0,1,Abortion should be banned! People are killing ...,F,A
1,1,I'm assuming we're talking about animals... eu...,T,A
2,1,I think gay marriage is just a normal as strai...,T,A
3,1,We need to keep them out of the country. They ...,F,A
4,1,I think it should be legalized and regulated l...,T,A
...,...,...,...,...
2495,500,I believe that abortion is a woman's right. A ...,T,A
2496,500,My opinion on Euthanasia is that it should not...,F,A
2497,500,Gay marriage is an atrocity to society. How ca...,F,A
2498,500,I think that migrants should come to the Unite...,T,A


In [84]:
hc = pd.read_csv('hcV3-stories.csv', sep=',', encoding='UTF-8')#encoding='ISO-8859-1')

mem = hc[hc['memType']!='retold']
mem = mem.dropna(subset=['story', 'memType'])
mem = mem[['story', 'memType']]
mem['memType'][mem['memType']=='recalled'] = 'T'
mem['memType'][mem['memType']=='imagined'] = 'F'
mem = mem.rename(columns={'story': 'sent', 'memType': 'labels'})
mem['type'] = 'B'

mem

,sent,labels,type
0,"Concerts are my most favorite thing, and my bo...",F,B
1,"The day started perfectly, with a great drive ...",T,B
2,It seems just like yesterday but today makes f...,F,B
3,"Five months ago, my niece and nephew were born...",T,B
4,About a month ago I went to burning man. I was...,F,B
...,...,...,...
6849,My dog was diagnosed with lymphoma a year ago ...,T,B
6850,"Over my vacation from my job, I went to Casper...",T,B
6851,This event was a birthday party for my nephew....,T,B
6852,This event occurred about two weeks ago. I was...,T,B


In [85]:
intent = pd.read_csv('sign_events_data_statements.csv', encoding="UTF-8")

intent.loc[intent['outcome_class']=='t', 'outcome_class'] = 'T'
intent.loc[intent['outcome_class']=='d', 'outcome_class'] = 'F'
intent['q1'] = intent['q1'].apply(lambda x: x.replace('\n', ''))
intent = intent.rename(columns={'q1': 'sent', 'outcome_class': 'labels'})
intent = intent[['sent', 'labels']]
intent['type'] = 'C'

intent

,sent,labels,type
0,My little brother is getting married next Satu...,T,C
1,I will be driving for 80-90 minutes across Lon...,T,C
2,We were planning to get dinner somewhere near ...,T,C
3,We are going to cannock chase with the mountai...,T,C
4,I will be getting up Tuesday morning to go int...,T,C
...,...,...,...
1635,I'm going on a walk with a friend of mine and ...,F,C
1636,I am going to a museum in Manchester that focu...,F,C
1637,We used to be housemates and always watched th...,F,C
1638,The RAF 100 Bicycle Trail is a commemorative t...,F,C


In [86]:
k = 1500
seed = 69

'''
shuffled_df = decop.sample(frac=0.1, random_state=seed).copy()
sample_decop = shuffled_df.iloc[:k,].copy()
sample_decop.drop('ID', axis=1, inplace=True)
sample_decop

data = pd.concat([sample_decop, mem.iloc[:k,].copy(), intent.iloc[:k,].copy()],
                 ignore_index=True)
'''

decop.drop('ID', axis=1, inplace=True)

data = pd.concat([decop, mem, intent], ignore_index=True)
data_shuffle = data.sample(frac=1, random_state=seed).reset_index(drop=True)
#data = data.drop(columns=['type'])
data_shuffle['labels'] = data_shuffle['labels'].map({'T': 1, 'F': 0})
data_shuffle.rename(columns={'sent': 'text', 'labels': 'label', 'type': 'set'}, inplace=True)

data_shuffle.iloc[:k,]

,text,label,set
0,I believe a large part of the U.S. workforce i...,1,A
1,I am for immigration if it is done legally. I ...,1,A
2,I recently interviewed for an in house positio...,1,B
3,"On the weekend, when both me and my child are ...",0,C
4,"Dear diary,Two weeks ago today I was in pain m...",0,B
...,...,...,...
1495,Monday my daughter started ‚ÄúBig Girl School‚...,1,B
1496,I still can't believe it happened to me. I tho...,0,B
1497,Today was definitely not the best of days. Aft...,1,B
1498,"I went on a business trip to Albany, New York....",1,B


In [87]:
train_data, temp_data = train_test_split(data_shuffle, test_size=0.3, random_state=42)

In [88]:
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

In [89]:
train_data.to_json(f'train_data_{k}.json', orient='records', lines=True)

In [90]:
val_data.to_json(f'val_data_{k}.json', orient='records', lines=True)

In [91]:
test_data.to_json(f'test_data_{k}.json', orient='records', lines=True)

In [92]:
train_data.shape

(6765, 3)